In [325]:
# !pip install pyspellchecker
# !pip install langdetect
# !pip install demoji

In [326]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from spellchecker import SpellChecker
from langdetect import detect


from tqdm import tqdm

import nltk
import re
import demoji
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt') 

[nltk_data] Downloading package wordnet to /Users/janice/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/janice/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/janice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [327]:
df_google = pd.read_csv("../data/ChatGPT-play-reviews.csv", 
                 parse_dates=["at", "repliedAt"])

df_apple_1 = pd.read_csv("../data/all_apple_reviews-1_2023-11-09.csv", parse_dates=["date"])
df_apple_2 = pd.read_csv("../data/all_apple_reviews-2_2023-11-09.csv", parse_dates=["date"])

df_apple = pd.concat([df_apple_1, df_apple_2], join='inner')

df_apple["content"] = df_apple["title"] + ". " + df_apple["review"]
df_apple["Source"] = "Apple"

df_google = df_google.drop('reviewCreatedVersion', axis=1)
df_google["Source"] = "Google"

In [328]:
print(df_apple.columns, df_apple.shape)
print(df_google.columns, df_google.shape)

Index(['country', 'date', 'review', 'rating', 'isEdited', 'userName', 'title',
       'content', 'Source'],
      dtype='object') (28207, 9)
Index(['reviewId', 'userName', 'content', 'score', 'thumbsUpCount', 'at',
       'replyContent', 'repliedAt', 'appVersion', 'Source'],
      dtype='object') (30956, 10)


In [329]:
df_apple.groupby('country').count()

,date,review,rating,isEdited,userName,title,content,Source
country,,,,,,,,
AE,111,111,111,111,111,111,111,111
AL,8,8,8,8,8,8,8,8
AM,2,2,2,2,2,2,2,2
AO,14,14,14,14,14,14,14,14
AR,84,84,84,84,84,84,84,84
...,...,...,...,...,...,...,...,...
us,10931,10931,10931,10931,10931,10931,10931,10931
uy,13,13,13,13,13,13,13,13
vn,40,40,40,40,40,40,40,40


In [330]:
df_apple = df_apple.drop(
    columns=['country', 'title', 'review']).rename(
    {'date': 'at', 'review': 'content', 'rating': 'score'}, axis='columns')

In [331]:
df = pd.concat([df_apple, df_google], axis = 0)

In [332]:
# var for review received response
df["reply"] = np.where(df.replyContent.isnull(), 0, 1)
df['score'] = df['score'].astype(int)
df['Reviews'] = 1
df['Original content'] = df['content']
df["score_cat"] = np.where(df.score == 5, "positive", np.where(df.score == 4, "neutral", "negative"))

In [333]:
# generate new date features from at
df['at_ymd'] = df['at'].dt.strftime('%D')
# Create new column for year-quarter
df['at_q'] = df['at'].dt.quarter
# Create new column for year-month
df['at_ym'] = df['at'].dt.strftime('%Y-%m')
# Create new column for month
df['at_m'] = df['at'].dt.strftime('%B')
# Create new column for year-month
df['at_wd'] = df['at'].dt.strftime('%A')
df['at_w'] = df['at'].dt.isocalendar().week
#df['at'] = df['at'].dt.date

In [334]:
# display number of missing values per column
df.isna().sum()

at                      0
score                   0
isEdited            30956
userName                1
content                 0
Source                  0
reviewId            28207
thumbsUpCount       28207
replyContent        58731
repliedAt           58731
appVersion          33121
reply                   0
Reviews                 0
Original content        0
score_cat               0
at_ymd                  0
at_q                    0
at_ym                   0
at_m                    0
at_wd                   0
at_w                    0
dtype: int64

Cleaning Costumer Reviews: 
Remove URLs, emails, phone numbers & punctuations.
Remove tags, emojis, symbols & pictographs.
Remove stop words.
Convert to lowercase and lemmatization.
Duplicates removal.
Spell checking.
Non-English reviews removal.
Remove stop words. 

### Remove duplicates

In [335]:
df.shape

(59163, 21)

In [336]:
count_duplicates = df.duplicated().sum()
count_duplicates

14075

In [321]:
df.sort_values('at', ascending=False).drop_duplicates(inplace=True, keep='first')
df.shape

(59163, 21)

In [337]:
count_duplicates = df.duplicated(subset=['content', 'userName']).sum()
count_duplicates

14078

In [342]:
df[df.duplicated(keep=False) == True].sort_values('content')

,at,score,isEdited,userName,content,Source,reviewId,thumbsUpCount,replyContent,repliedAt,...,reply,Reviews,Original content,score_cat,at_ymd,at_q,at_ym,at_m,at_wd,at_w


In [341]:
df.drop_duplicates(subset=['content', 'userName'],inplace=True)
df.shape

(45085, 21)

### Remove NAs

In [343]:
df.isna().sum()

at                      0
score                   0
isEdited            30954
userName                1
content                 0
Source                  0
reviewId            14131
thumbsUpCount       14131
replyContent        44653
repliedAt           44653
appVersion          19045
reply                   0
Reviews                 0
Original content        0
score_cat               0
at_ymd                  0
at_q                    0
at_ym                   0
at_m                    0
at_wd                   0
at_w                    0
dtype: int64

### Remove emojis and symbols, standardize mentions of ChatGPT and OpenAI

In [344]:
def pre_process(text):

    # remove emojis
    emoji_pattern = re.compile("["
         u"\U00002700-\U000027BF"  # Dingbats
         u"\U0001F600-\U0001F64F"  # Emoticons
         u"\U00002600-\U000026FF"  # Miscellaneous Symbols
         u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
         u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
         u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
         u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                       "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item, '')
    
    # remove all characters that are not alphanumeric
    #text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # remove symbols
    symbol_pattern = re.compile(r'[@#$%^&*()_+{}\[\]"\<>,/\\|`~]+')
    text = symbol_pattern.sub(r'', text)

    # remove - 
    dash_pattern = re.compile(r'-+')
    text = dash_pattern.sub(r'', text)

    #split the string into separate tokens
    tokens = re.split(r"\s+",text)

    # normalise all words into lowercase
    text = " ".join([t.lower() for t in tokens])

    # standardize
    text = text.replace("chat gpt", "chatgpt")
    text = text.replace("open ai", "openai")

    # return final list of tokens
    return text

In [345]:
tqdm.pandas()
df['content'] = df['content'].progress_apply(pre_process)

  0%|          | 0/45085 [00:00<?, ?it/s]

100%|██████████| 45085/45085 [00:20<00:00, 2228.38it/s] 


In [346]:
df['content'].tail(90)

30866         yo tengo
30867    thank chatgpt
30868       1st review
30869         just wow
30870     تطبيق احتراف
             ...      
30951             ️️️️
30952             ️️️️
30953             ️️️️
30954                ️
30955                5
Name: content, Length: 90, dtype: object

In [347]:
# test_text = "Amaznig and extremely handy app for many uses.... 🤍 it's like an extension of one's fingers ⭐️⭐️. #ChatGPT Chat GPT OpenAI Open AI HTML Google"

In [348]:
#corrected_text = pre_process(test_text)
#print(test_text)
#print(corrected_text)

In [349]:
# def remove_hashtags(text): 
#     hashtag_pattern = re.compile(r'#\S+')
#     return hashtag_pattern.sub('', text)
# df['content'] = df['content'].apply(remove_hashtags)

### Detect language 

In [350]:
def detect_language(comment):
    try:
        return detect(comment)
    except:
        return 'unknown' 

In [351]:
tqdm.pandas()
df['detected_language'] = df['content'].progress_apply(detect_language)

100%|██████████| 45085/45085 [02:35<00:00, 290.36it/s]


In [352]:
df['detected_language'].unique()

array(['en', 'fr', 'pt', 'es', 'so', 'unknown', 'cy', 'sw', 'it', 'cs',
       'tr', 'ru', 'sv', 'nl', 'tl', 'fi', 'zh-cn', 'af', 'de', 'no',
       'id', 'et', 'sl', 'ca', 'da', 'ro', 'lt', 'sk', 'hu', 'bg', 'vi',
       'sq', 'ar', 'hr', 'lv', 'pl', 'th', 'ur', 'uk', 'fa', 'ko', 'el',
       'he', 'ja', 'mr', 'zh-tw', 'bn', 'ml', 'ta', 'hi', 'ne', 'te',
       'gu', 'kn', 'mk'], dtype=object)

In [353]:
lang = df[df['detected_language'] == "en"]
lang.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30300 entries, 0 to 30923
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   at                 30300 non-null  datetime64[ns]
 1   score              30300 non-null  int64         
 2   isEdited           8950 non-null   object        
 3   userName           30299 non-null  object        
 4   content            30300 non-null  object        
 5   Source             30300 non-null  object        
 6   reviewId           21350 non-null  object        
 7   thumbsUpCount      21350 non-null  float64       
 8   replyContent       390 non-null    object        
 9   repliedAt          390 non-null    datetime64[ns]
 10  appVersion         18076 non-null  object        
 11  reply              30300 non-null  int64         
 12  Reviews            30300 non-null  int64         
 13  Original content   30300 non-null  object        
 14  score_cat  

In [354]:
#df.to_csv("../data/chatgpt_short_clean_all_languages.csv")

### Split into short (review-wise) and long format (sentence-wise)

In [355]:
df_en = df[df['detected_language'] == "en"]

In [383]:
df_en_long = df_en

# Function to apply nltk.sent_tokenize() to a text column and create a new row for each sentence
def tokenize_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

# Apply the function to the 'text_column' and explode the list of rows into individual rows
tqdm.pandas()
df_en_long['sentence'] = df_en_long.content.progress_apply(tokenize_sentences)
df_en_long = df_en_long.explode('sentence')

  0%|          | 0/30300 [00:00<?, ?it/s]

100%|██████████| 30300/30300 [00:00<00:00, 37675.94it/s]
/var/folders/vl/wpx53sjn0nv5cmny0jckrfjr0000gn/T/ipykernel_13269/2461770427.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_long['sentence'] = df_en_long.content.progress_apply(tokenize_sentences)


In [384]:
df_en_long.sort_values('Source').sort_index(ascending=False).head(100)

,at,score,isEdited,userName,content,Source,reviewId,thumbsUpCount,replyContent,repliedAt,...,Original content,score_cat,at_ymd,at_q,at_ym,at_m,at_wd,at_w,detected_language,sentence
30923,2023-07-25 17:09:58,5,NaN,Rohan Dafare,hum first,Google,beb0b506-732d-4d42-888b-1485a3958ab2,0.0,NaN,NaT,...,Hum First,positive,07/25/23,3,2023-07,July,Tuesday,30,en,hum first
30920,2023-07-27 17:53:37,5,NaN,asac schrader,hell yeah,Google,09b3f8d3-cd42-4adf-8e61-ca70865fc853,0.0,NaN,NaT,...,Hell yeah,positive,07/27/23,3,2023-07,July,Thursday,30,en,hell yeah
30918,2023-07-25 17:15:19,5,NaN,mostafijur rahman,usually app,Google,0530373c-1bfc-45d2-9dec-9fc0bb9cff4d,0.0,NaN,NaT,...,Usually app,positive,07/25/23,3,2023-07,July,Tuesday,30,en,usually app
30916,2023-07-25 17:08:33,5,NaN,Sarvesh Soni,first downloader,Google,2ecc7803-920c-4f95-8b15-db9c7b1caa8a,0.0,NaN,NaT,...,First downloader,positive,07/25/23,3,2023-07,July,Tuesday,30,en,first downloader
30912,2023-07-25 17:47:09,5,NaN,Yeamin Hossain (Limon),first review,Google,6bed36cc-bc58-479e-b395-6644d15adeb8,0.0,NaN,NaT,...,First Review,positive,07/25/23,3,2023-07,July,Tuesday,30,en,first review
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30533,2023-07-25 17:27:14,5,NaN,Fahad A. Faiz,first?,Google,a23d1be8-c17c-43c2-944c-347168eeb408,0.0,NaN,NaT,...,First? 🤔,positive,07/25/23,3,2023-07,July,Tuesday,30,en,first?
30527,2023-07-26 16:23:57,1,NaN,Saroj Kumar,no downloading,Google,ab5a6bd7-be2b-4422-b718-3909276cbb20,0.0,NaN,NaT,...,No downloading,negative,07/26/23,3,2023-07,July,Wednesday,30,en,no downloading
30525,2023-07-26 09:20:00,1,NaN,RAJ RAJA,low quantity,Google,5682b85e-3e01-4335-835d-250220e97305,0.0,NaN,NaT,...,Low quantity,negative,07/26/23,3,2023-07,July,Wednesday,30,en,low quantity
30524,2023-08-31 15:58:43,1,NaN,Sudha Suresh kumar,not responding,Google,09ac9623-73e5-42ba-82bf-c73c8f32e21f,0.0,Thank you for your feedback. Is this problem still happening?\n,2023-09-01 22:49:51,...,Not responding,negative,08/31/23,3,2023-08,August,Thursday,35,en,not responding


In [385]:
df_en_long.shape

(64984, 23)

### Spell checking

In [386]:
spell = SpellChecker()
exceptions = ["chatgpt", "openai", "gpt", "html", "css", "javascript", "microsoft", "elon"]

In [387]:
def correct_spelling(text): 
    words = text.split()
    corrected_text = []
    for word in words:
        if word in exceptions:
            corrected_text.append(word)
        else: 
            corrected_word = spell.correction(word)
            if corrected_word is not None: 
                corrected_text.append(corrected_word)
            else:
                corrected_text.append(word)
    corrected_text = " ".join(corrected_text)
    return corrected_text

In [388]:
# corrected_text2 = correct_spelling(corrected_text)
# print(test_text)
# print(corrected_text)
# print(corrected_text2)

In [389]:
tqdm.pandas()
df_en['content'] = df_en['content'].progress_apply(correct_spelling)
tqdm.pandas()
df_en_long['sentence'] = df_en_long['sentence'].progress_apply(correct_spelling)

  0%|          | 0/30300 [00:00<?, ?it/s]

100%|██████████| 30300/30300 [39:18<00:00, 12.85it/s]  
/var/folders/vl/wpx53sjn0nv5cmny0jckrfjr0000gn/T/ipykernel_13269/3856361211.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en['content'] = df_en['content'].progress_apply(correct_spelling)
100%|██████████| 64984/64984 [38:17<00:00, 28.29it/s]   


### Save dataframes

In [390]:
df_en.to_csv("../data/chatgpt_short_clean_combined_en.csv")
df_en_long.to_csv("../data/long/chatgpt_clean_combined_en.csv")

In [391]:
df_en_long

,at,score,isEdited,userName,content,Source,reviewId,thumbsUpCount,replyContent,repliedAt,...,Original content,score_cat,at_ymd,at_q,at_ym,at_m,at_wd,at_w,detected_language,sentence
0,2023-10-30 21:26:19,5,False,Andrea Pepkolaj,the best. the best help you cant get you just need to be a bit of protective,Apple,NaN,NaN,NaN,NaT,...,The best. The best help you cant get you just need to be a bit of protective,positive,10/30/23,4,2023-10,October,Monday,44,en,the best
0,2023-10-30 21:26:19,5,False,Andrea Pepkolaj,the best. the best help you cant get you just need to be a bit of protective,Apple,NaN,NaN,NaN,NaT,...,The best. The best help you cant get you just need to be a bit of protective,positive,10/30/23,4,2023-10,October,Monday,44,en,the best help you cant get you just need to be a bit of protective
1,2023-05-29 22:57:48,4,False,gjoncari.jurgen,can’t edit a question. can’t edit a question like in the browser. also i like the complete black background but the letters should be grayer to not have a too high contrast with it.,Apple,NaN,NaN,NaN,NaT,...,"Can’t edit a question. Can’t edit a question like in the browser. Also, I like the complete black background but the letters should be grayer to not have a too high contrast with it.",neutral,05/29/23,2,2023-05,May,Monday,22,en,can't edit a question
1,2023-05-29 22:57:48,4,False,gjoncari.jurgen,can’t edit a question. can’t edit a question like in the browser. also i like the complete black background but the letters should be grayer to not have a too high contrast with it.,Apple,NaN,NaN,NaN,NaT,...,"Can’t edit a question. Can’t edit a question like in the browser. Also, I like the complete black background but the letters should be grayer to not have a too high contrast with it.",neutral,05/29/23,2,2023-05,May,Monday,22,en,can't edit a question like in the browser
1,2023-05-29 22:57:48,4,False,gjoncari.jurgen,can’t edit a question. can’t edit a question like in the browser. also i like the complete black background but the letters should be grayer to not have a too high contrast with it.,Apple,NaN,NaN,NaN,NaT,...,"Can’t edit a question. Can’t edit a question like in the browser. Also, I like the complete black background but the letters should be grayer to not have a too high contrast with it.",neutral,05/29/23,2,2023-05,May,Monday,22,en,also i like the complete black background but the letters should be grayer to not have a too high contrast with it
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30912,2023-07-25 17:47:09,5,NaN,Yeamin Hossain (Limon),first review,Google,6bed36cc-bc58-479e-b395-6644d15adeb8,0.0,NaN,NaT,...,First Review,positive,07/25/23,3,2023-07,July,Tuesday,30,en,first review
30916,2023-07-25 17:08:33,5,NaN,Sarvesh Soni,first downloader,Google,2ecc7803-920c-4f95-8b15-db9c7b1caa8a,0.0,NaN,NaT,...,First downloader,positive,07/25/23,3,2023-07,July,Tuesday,30,en,first downloaded
30918,2023-07-25 17:15:19,5,NaN,mostafijur rahman,usually app,Google,0530373c-1bfc-45d2-9dec-9fc0bb9cff4d,0.0,NaN,NaT,...,Usually app,positive,07/25/23,3,2023-07,July,Tuesday,30,en,usually app
30920,2023-07-27 17:53:37,5,NaN,asac schrader,hell yeah,Google,09b3f8d3-cd42-4adf-8e61-ca70865fc853,0.0,NaN,NaT,...,Hell yeah,positive,07/27/23,3,2023-07,July,Thursday,30,en,hell yeah
